In [1]:
import math
import os
import random
import time
import unicodedata  # 자음/모음 분리현상 방지용 모듈
import urllib
import urllib.request
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

DB = pd.read_excel("C:/Users/PC/Desktop/부산광역시 RPA/기업DB_Kisline_양식.xlsx")
company_list = pd.read_excel("C:/Users/PC/Desktop/부산광역시 RPA/기업List.xlsx")
company_name = company_list.회사명
#company_ceo = company_list.대표자

#"주식회사" 를 (주)로 변경
company_name2 = []

for i in company_name:
    temp = i.replace("주식회사", "(주)")
    company_name2.append(temp)

#아이디 비번
v_id = input("1. ID를 입력하세요: ")
v_passwd = input("2. 비밀번호를 입력하세요: ")
#v_company_name = input("3. 회사명을 입력하세요: ")

#저장할 파일 경로
f_dir = "C://Users//PC//Desktop//"
#저장할 파일 이름
sec_name = "기업"
query_txt = "KISLINE"

n = time.localtime()
s1 = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s1+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s1+'-'+query_txt+'-'+sec_name)

ff_dir = f_dir+s1+'-'+query_txt+'-'+sec_name
fc_name = f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.csv'

#driver 세팅
chrome_driver_path = 'C:/Users/PC/Desktop/부산광역시 RPA/chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("disable-blink-features=AutomationControlled")
#options.add_argument('headless')
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
args = ["hide_console", ]
#options.add_argument("--disable-extensions");
options.add_argument("disable-infobars");
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(chrome_driver_path, options=options, service_args=args)

#크롤링 시작
base_url = 'https://www.kisline.com/cm/CM0100M00GE00.nice' 
driver.set_window_size(1124, 850)
driver.get(base_url)
#time.sleep(5)
driver.maximize_window()
#WebDriverWait(driver, 200).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div[2]/section[2]/div/div[1]/div[2]/div[1]/div[1]/div/module/div[1]")))
time.sleep(random.randrange(1,5))

#팝업
driver.find_element(By.XPATH,'//*[@id="userGuideCookie"]').click()
driver.find_element(By.XPATH,'//*[@id="layout_guide"]/div[2]/a/span').click()
driver.find_element(By.ID,'btn_login').click()
time.sleep(1)

#id
eid = driver.find_element(By.NAME,'lgnuid')
for a in v_id:
    eid.send_keys(a)
    time.sleep(0.3)
#passwd
epwd = driver.find_element(By.CLASS_NAME,'pwType').click()
driver.find_element(By.CLASS_NAME,'pwType').send_keys(v_passwd)
time.sleep(1)
driver.find_element(By.XPATH,'//*[@id="loginForm"]/div/fieldset/a/span').click()
time.sleep(1)

#팝업
driver.find_element(By.XPATH,'//*[@id="close_today"]').click()
driver.find_element(By.XPATH,'//*[@id="closeAndSetcookie"]').click()

#기업 검색
#ecpn = driver.find_element(By.ID,'q').click()
#driver.find_element(By.ID,'q').send_keys(v_company_name)
#driver.find_element(By.ID, "searchView").click()
#time.sleep(1)
#driver.find_element(By.LINK_TEXT, v_company_name).click()

#기업코드
#kiscode = basic_info_table_row[0].find("td").get_text().split(")", -1)[-2].split("(", -1)[-1]
#기업명
#company_name = basic_info_table_row[0].find("td").get_text().rsplit("(", 1)[0].strip()

#텍스트 추출 함수
def get_text(soup):
    return soup.get_text().replace('\n', ' ').strip()

count = 1

#기업 검색
for name, num in zip(company_name2, range(0, len(company_name2))):
    try:
        search = driver.find_element(By.ID,'q')
        search.clear()
        search.send_keys(name)
        driver.find_element(By.ID, "searchView").click()
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        #보고서 검색 건수
        report_info = []
        report_info_table = soup.find_all("div", "sc_result")[4]
        x = get_text(report_info_table.find("em"))

        report_info.append(x.replace("(", "").replace(")", ""))
        
        #뉴스 검색 건수
        news_info = []
        news_info_table = soup.find_all("div", "sc_result")[5]
        z = get_text(news_info_table.find("em"))

        news_info.append(z.replace("(", "").replace(")", ""))
        
        #상단 기업 클릭
        driver.find_element(By.NAME, 'overViewOpen').click()
        time.sleep(0.5)
        
        #파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        #basic_info
        basic_info = []
        basic_info_table = soup.find_all("tbody")[0]
        basic_info_table_row = basic_info_table.find_all("tr")

        for i in range(0, 14):
            try:
                if i == 6:
                    a = basic_info_table_row[i].find("td").get_text().replace("\n", "").replace(" ","").strip()
                    a = a.split(")")[0].replace("(", "")
                    basic_info.append(a)
                elif i == 7:
                    a = basic_info_table_row[i - 1].find("td").get_text().replace("\n", "").replace(" ","").strip()
                    a = a.split(")")[1]
                    basic_info.append(a)
                elif i == 13:
                    a = basic_info_table_row[i - 1].find("td").get_text().replace("\n", "").replace(" ","").strip()
                    a = a.replace(a[-8:], "")
                    basic_info.append(a)    
                elif i > 7:
                    a = basic_info_table_row[i - 1].find("td").get_text().replace("\n", "").replace(" ","").strip()
                    basic_info.append(a)
                else:
                    a = basic_info_table_row[i].find("td").get_text().replace("\n", "").replace(" ","").strip()
                    basic_info.append(a)
            except:
                a = np.nan
                basic_info.append(a)

        #kpi_info
        kpi_info = []
        kpi_info_table = soup.find_all("tbody")[1]
        kpi_info_table_column = kpi_info_table.find_all("tr")

        for i in range(0, 5):
            for j in range(1, 8):
                try:
                    b = kpi_info_table_column[i].find_all("td")[j].get_text().replace("\n", "").strip()
                    kpi_info.append(b)
                except:
                    b = "-"
                    kpi_info.append(b)

        #경영진형황
        manage_info = []
        manage_info_table = soup.find_all("tbody")[2]
        manage_info_table_row = manage_info_table.find_all("tr")

        for i in range(0, 3):
            for j in range(0, 2):
                try:
                    c = manage_info_table_row[i].find_all("td")[j].get_text().replace("/n", "").strip()
                    manage_info.append(c)
                except:
                    c = "-"
                    manage_info.append(c)

        #주요주주 현황
        stockholder_info = []
        stockholder_info_table = soup.find_all("tbody")[3]
        stockholder_info_table_row = stockholder_info_table.find_all("tr")

        for i in range(0, 3):
            for j in range(0, 3):
                try:
                    d = stockholder_info_table_row[i].find_all("td")[j].get_text().replace("/n", "").strip()
                    if d == "조회된 데이터가 없습니다.":
                        d = "-"
                    stockholder_info.append(d) 
                except:
                    d = "-"
                    stockholder_info.append(d)

        #관계회사 현황
        relatedcompany_info = []
        relatedcompany_info_table = soup.find_all("tbody")[4]
        relatedcompany_info_table_row = relatedcompany_info_table.find_all("tr")

        for i in range(0, 3):
            for j in range(0, 3):
                try:
                    e = relatedcompany_info_table_row[i].find_all("td")[j].get_text().replace("/n", "").strip()
                    if e == "조회된 데이터가 없습니다.":
                        e = "-"
                    relatedcompany_info.append(e) 
                except:
                    e = "-"
                    relatedcompany_info.append(e)

        #등급현황
        rating_info = []
        rating_info_table = soup.find("div", "overview_area03")
        rating_info_table_row = rating_info_table.find_all("div")

        for i in range(0, 3):
            try:
                if i == 0: 
                    f = rating_info_table_row[i].find("strong").get_text().replace("\n", "").strip()
                    rating_info.append(f)    
                elif i == 1:
                    f = rating_info_table_row[i+1].find("strong").get_text().replace("\n", "").strip()
                    rating_info.append(f)
                else:
                    f = rating_info_table_row[5].find("dt").get_text().split(" ")[0].replace("\n", "").strip()
                    rating_info.append(f)
            except:
                f = "-"
                rating_info.append(f)
            
        #주요 매출처
        sale_info = []
        sale_info_table = soup.find_all("tbody")[5]
        sale_info_table_row = sale_info_table.find_all("tr")

        for i in range(0, 8):
            try:
                h = sale_info_table_row[0].find_all("td")[i].get_text().replace("/n", "").strip()
                if h == "조회된 데이터가 없습니다.":
                    h = "-"
                sale_info.append(h) 
            except:
                h = "-"
                sale_info.append(h)

        #주요 매입처
        purchase_info = []
        purchase_info_table = soup.find_all("tbody")[6]
        purchase_info_table_row = purchase_info_table.find_all("tr")

        for i in range(0, 8):
            try:
                l = purchase_info_table_row[0].find_all("td")[i].get_text().replace("/n", "").strip()
                if l == "조회된 데이터가 없습니다.":
                    l = "-"
                purchase_info.append(l) 
            except:
                l = "-"
                purchase_info.append(l)

        #소속산업 현황
        industy_info = []
        industy_info_table = soup.find("div", "overview_area04")
        industy_info_table_row = industy_info_table.find("div", "left_area").find_all("tr")

        for i in range(0, 4):
            try:
                m = industy_info_table_row[i].find("td").get_text().replace("\n", "").strip()
                industy_info.append(m)
            except:
                m = "-"
                industy_info.append(m)

        #산업내 경쟁분석(국내)
        domestic_info = []
        cnt = soup.find_all("tbody")
        if len(cnt) == 12:
            domestic_info_table = soup.find_all("tbody")[10]
        else:
            domestic_info_table = soup.find_all("tbody")[9]
            
        domestic_info_table_row = domestic_info_table.find_all("tr")

        for i in range(0, 4):
            try:
                if i == 3:
                    n = domestic_info_table_row[i - 1].find_all("td")[7].get_text().replace("\n", "").strip()
                    domestic_info.append(n)            
                else:
                    n = domestic_info_table_row[i].find_all("td")[6].get_text().replace("\n", "").strip()
                    domestic_info.append(n)           
            except:
                n = "-"
                domestic_info.append(n)

        #산업내 경쟁분석(지역)
        area_info = []
        if len(cnt) == 12: 
            area_info_table = soup.find_all("tbody")[11]
        else:
            area_info_table = soup.find_all("tbody")[10]
            
        area_info_table_row = area_info_table.find_all("tr")

        for i in range(0, 4):
            try:
                if i == 3:
                    o = area_info_table_row[i - 1].find_all("td")[7].get_text().replace("\n", "").strip()
                    area_info.append(o)            
                else:
                    o = area_info_table_row[i].find_all("td")[6].get_text().replace("\n", "").strip()
                    area_info.append(o)           
            except:
                o = "-"
                area_info.append(o)

        #산업 내 위치
        location_info = []
        location_info_table = soup.find("div", "overview_area04")
        location_info_table_row = location_info_table.find("div", "right_area")

        for i in range(0, 5):
            try:
                p = location_info_table_row.find_all("li")[i].find("em").get_text().replace("\n", "").strip()
                location_info.append(p)
            except:
                p = "-"
                location_info.append(p)


        #종합의견
        opinion_info = []
        opinion_info_table = soup.find("div", "opinion_detail")
        opinion_info_table_row = opinion_info_table.find_all("dl")

        for i in range(0, 30):
            try:
            #안정성 분석
                if i == 0:
                    q = get_text(opinion_info_table_row[2].find_all("dd")[1])
                    r = q.split(" ")[2].replace("이며,", "")
                    opinion_info.append(r)
                elif i == 1:
                    q = get_text(opinion_info_table_row[2].find_all("dd")[1])
                    q = q.replace(q[q.find("["):q.find("]") + 1], "")
                    r = q.split(" ")[8].replace("로", "")
                    opinion_info.append(r)
                elif i == 2:
                    q = get_text(opinion_info_table_row[2].find_all("dd")[1])
                    q = q.replace(q[q.find("["):q.find("]") + 1], "")
                    r = q.split(" ")[12].replace("[","").replace("]","").replace("합니다.","")
                    opinion_info.append(r)
                elif i == 3:
                    q = get_text(opinion_info_table_row[2].find_all("dd")[2])
                    r =q.split(" ")[1].replace("이며,","")
                    opinion_info.append(r)
                elif i == 4:
                    q = get_text(opinion_info_table_row[2].find_all("dd")[2])
                    q = q.replace(q[q.find("["):q.find("]") + 1], "")
                    r = q.split(" ")[7].replace("로","")
                    opinion_info.append(r)
                elif i == 5:
                    q = get_text(opinion_info_table_row[2].find_all("dd")[2])
                    q = q.replace(q[q.find("["):q.find("]") + 1], "")
                    r = q.split(" ")[11].replace("[","").replace("]","").replace("합니다.","")
                    opinion_info.append(r)
                #유동성 분석
                elif i == 6:
                    q = get_text(opinion_info_table_row[3].find_all("dd")[1])
                    r = q.split(" ")[2].replace("이고,","")
                    opinion_info.append(r)
                elif i == 7:
                    q = get_text(opinion_info_table_row[3].find_all("dd")[1])
                    r = q.split(" ")[8].replace("로","")
                    opinion_info.append(r)
                elif i == 8:
                    q = get_text(opinion_info_table_row[3].find_all("dd")[1])
                    r = q.split(" ")[12].replace("[","").replace("]","").replace("하며,","")
                    opinion_info.append(r)
                elif i == 9:
                    q = get_text(opinion_info_table_row[3].find_all("dd")[1])
                    r = q.split(" ")[14].replace("이고","")
                    opinion_info.append(r)
                elif i == 10:
                    q = get_text(opinion_info_table_row[3].find_all("dd")[1])
                    r = q.split(" ")[17].replace("이므로","")
                    opinion_info.append(r)
                elif i == 11:
                    q = get_text(opinion_info_table_row[3].find_all("dd")[1])
                    r = q.split(" ")[21].replace("[","").replace("]","").replace("한","")
                    opinion_info.append(r)
                #수익성 분석
                elif i == 12:
                    q = get_text(opinion_info_table_row[4].find_all("dd")[1])
                    r = q.split(" ")[2].replace("이며,","")
                    opinion_info.append(r)
                elif i == 13:
                    q = get_text(opinion_info_table_row[4].find_all("dd")[1])
                    r = q.split(" ")[8].replace("로","")
                    opinion_info.append(r)
                elif i == 14:
                    q = get_text(opinion_info_table_row[4].find_all("dd")[1])
                    r = q.split(" ")[12].replace("[","").replace("]","").replace("한","")
                    opinion_info.append(r)
                elif i == 15:
                    q = get_text(opinion_info_table_row[4].find_all("dd")[1])
                    r = q.split(" ")[15].replace(",","")
                    opinion_info.append(r)
                elif i == 16:
                    q = get_text(opinion_info_table_row[4].find_all("dd")[1])
                    r = q.split(" ")[18].replace("로","")
                    opinion_info.append(r)
                elif i == 17:
                    q = get_text(opinion_info_table_row[4].find_all("dd")[1])
                    r = q.split(" ")[22].replace("[","").replace("]","").replace("한","")
                    opinion_info.append(r)
                #성장성 분석
                elif i == 18:
                    q = get_text(opinion_info_table_row[5].find_all("dd")[0])
                    r = q.split(" ")[4].replace("이며,","")
                    opinion_info.append(r)
                elif i == 19:
                    q = get_text(opinion_info_table_row[5].find_all("dd")[0])
                    r = q.split(" ")[10].replace("입니다.","")
                    opinion_info.append(r)
                elif i == 20:
                    q = get_text(opinion_info_table_row[5].find_all("dd")[1])
                    r = q.split(" ")[5].replace("로","")
                    opinion_info.append(r)
                elif i == 21:
                    q = get_text(opinion_info_table_row[5].find_all("dd")[1])
                    r = q.split(" ")[9].replace("[","").replace("]","").replace("하며,","")
                    opinion_info.append(r)
                elif i == 22:
                    q = get_text(opinion_info_table_row[5].find_all("dd")[1])
                    r = q.split(" ")[12].replace("로","").strip()
                    opinion_info.append(r)
                elif i == 23:
                    q = get_text(opinion_info_table_row[5].find_all("dd")[1])
                    r = q.split(" ")[15].replace("[","").replace("]","").replace("한","")
                    opinion_info.append(r)
                #활동성 분석
                elif i == 24:
                    q = get_text(opinion_info_table_row[6].find_all("dd")[0])
                    r = q.split(" ")[5].replace("이며,","").strip()
                    opinion_info.append(r)
                elif i == 25:
                    q = get_text(opinion_info_table_row[6].find_all("dd")[0])
                    r = q.split(" ")[10].replace("입니다.","").strip()
                    opinion_info.append(r)
                elif i == 26:
                    q = get_text(opinion_info_table_row[6].find_all("dd")[1])
                    r = q.split(" ")[5].replace("로","").strip()
                    opinion_info.append(r)
                elif i == 27:
                    q = get_text(opinion_info_table_row[6].find_all("dd")[1])
                    r = q.split(" ")[11].replace("[","").replace("]","").replace("하며,","")
                    opinion_info.append(r)
                elif i == 28:
                    q = get_text(opinion_info_table_row[6].find_all("dd")[1])
                    r = q.split(" ")[14].replace("로","")
                    opinion_info.append(r)
                elif i == 29:
                    q = get_text(opinion_info_table_row[6].find_all("dd")[1])
                    r = q.split(" ")[19].replace("[","").replace("]","").replace("한","")
                    opinion_info.append(r)
            except:
                r = "-"
                opinion_info.append(r)            
                
        #DB에 저장
        for i in range(0, 14):
            DB.loc[num + 4, DB.columns[i]] = basic_info[i]
        for i, j in zip(range(14, 49), range(0, 36)):
            DB.loc[num + 4, DB.columns[i]] = kpi_info[j]
        for i, j in zip(range(49, 55), range(0, 7)):
            DB.loc[num + 4, DB.columns[i]] = manage_info[j]
        for i, j in zip(range(55, 64), range(0, 10)):
            DB.loc[num + 4, DB.columns[i]] = stockholder_info[j]
        for i, j in zip(range(64, 73), range(0, 10)):
            DB.loc[num + 4, DB.columns[i]] = relatedcompany_info[j]
        for i, j in zip(range(73, 76), range(0, 4)):
            DB.loc[num + 4, DB.columns[i]] = rating_info[j]
        for i, j in zip(range(76, 84), range(0, 8)):
            DB.loc[num + 4, DB.columns[i]] = sale_info[j]
        for i, j in zip(range(84, 92), range(0, 8)):
            DB.loc[num + 4, DB.columns[i]] = purchase_info[j]
        for i, j in zip(range(92, 96), range(0, 5)):
            DB.loc[num + 4, DB.columns[i]] = industy_info[j]
        for i, j in zip(range(96, 100), range(0, 5)):
            DB.loc[num + 4, DB.columns[i]] = domestic_info[j]
        for i, j in zip(range(100, 104), range(0, 5)):
            DB.loc[num + 4, DB.columns[i]] = area_info[j]
        for i, j in zip(range(104, 109), range(0, 6)):
            DB.loc[num + 4, DB.columns[i]] = location_info[j]
        for i, j in zip(range(109, 139), range(0,31)):
            DB.loc[num + 4, DB.columns[i]] = opinion_info[j]
        for i, j in zip(range(139, 140), range(0,1)):
            DB.loc[num + 4, DB.columns[i]] = report_info[j]
        for i, j in zip(range(140, 141), range(0,1)):
            DB.loc[num + 4, DB.columns[i]] = news_info[j]
        time.sleep(1)
            
        #로그인 연장 
        driver.find_element(By.XPATH,'//*[@id="header"]/div[1]/ul/li[2]/a').click()
    except:
        pass
    
    #진행현황
    if count % 100 == 0:
        print(count, "개 진행중")
    count += 1
    
driver.close()

DB.to_csv(fc_name, encoding = "utf-8-sig", index = False)

c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=107.0.5304.121)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C18CE3+2067683]
	Ordinal0 [0x00BACBC1+1625025]
	Ordinal0 [0x00AAB7CD+571341]
	Ordinal0 [0x00A92E1A+470554]
	Ordinal0 [0x00AFAA2B+895531]
	Ordinal0 [0x00B0ACC6+961734]
	Ordinal0 [0x00AF7156+880982]
	Ordinal0 [0x00ACFF1D+720669]
	Ordinal0 [0x00AD0F5F+724831]
	GetHandleVerifier [0x00ECF162+2771330]
	GetHandleVerifier [0x00EC1145+2713957]
	GetHandleVerifier [0x00CACC0A+533546]
	GetHandleVerifier [0x00CAB970+528784]
	Ordinal0 [0x00BB4F4C+1658700]
	Ordinal0 [0x00BB98B8+1677496]
	Ordinal0 [0x00BB99A5+1677733]
	Ordinal0 [0x00BC4CC4+1723588]
	BaseThreadInitThunk [0x76B76739+25]
	RtlGetFullPathName_UEx [0x77988FD2+1218]
	RtlGetFullPathName_UEx [0x77988F9D+1165]


In [2]:
import pandas as pd

DB = pd.read_excel("C:/Users/PC/Desktop/부산광역시 RPA/기업DB_Kisline_양식.xlsx")
DB

,Basic,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,검색 후 첫화면에서,Unnamed: 140
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,종합의견(활동성 분석),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,기업명(기업코드),대표자명,설립일자,사업자번호,법인번호,기업형태,업종코드,표준산업분류 10차,그룹명,종업원수,...,속한산업 총자산증가율,상대적,총자산회전율,속한산업 총자산회전율,상대적,매출채권회전율,속한산업 매출채권회전율,상대적,보고서 검색 건 수,뉴스검색 건 수
